<a href="https://colab.research.google.com/github/phamduyaaaa/Robot-Kinematic/blob/main/Visai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import os
import cv2

# Khởi tạo thông số robot
class DifferentialRobot:
    def __init__(self, start_pos, width):
        self.x, self.y = start_pos
        self.theta = 0
        self.vl = 0.01
        self.vr = 0.01
        self.w = width
        self.max_speed = 0.02
        self.min_speed = -0.02
        self.trail = []

    def update(self, dt):
        # Giới hạn vận tốc
        self.vl = np.clip(self.vl, self.min_speed, self.max_speed)
        self.vr = np.clip(self.vr, self.min_speed, self.max_speed)

        v = (self.vl + self.vr) / 2
        omega = (self.vr - self.vl) / self.w

        self.x += v * math.cos(self.theta) * dt
        self.y += v * math.sin(self.theta) * dt
        self.theta += omega * dt
        self.theta %= 2 * math.pi

        self.trail.append((self.x, self.y))
        if len(self.trail) > 1000:
            self.trail.pop(0)

# Hàm vẽ robot
def draw_frame(robot, frame_num):
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.set_xlim(0, 4)
    ax.set_ylim(0, 2)
    ax.set_aspect('equal')
    ax.set_title(f'Frame {frame_num}')

    # Vẽ quỹ đạo
    if len(robot.trail) > 1:
        trail = np.array(robot.trail)
        ax.plot(trail[:, 0], trail[:, 1], 'r-')

    # Vẽ robot
    robot_size = 0.1
    rect = patches.Rectangle(
        (robot.x - robot_size/2, robot.y - robot_size/2),
        robot_size, robot_size,
        angle=np.degrees(robot.theta),
        linewidth=2, edgecolor='blue', facecolor='cyan', zorder=5)
    ax.add_patch(rect)

    # Vẽ trục X (đỏ), Y (xanh lá)
    ax.arrow(robot.x, robot.y,
             0.2 * math.cos(robot.theta),
             0.2 * math.sin(robot.theta),
             head_width=0.05, color='red')
    ax.arrow(robot.x, robot.y,
             0.2 * math.cos(robot.theta + math.pi / 2),
             0.2 * math.sin(robot.theta + math.pi / 2),
             head_width=0.05, color='green')

    filename = f"frame_{frame_num:04d}.png"
    plt.savefig(filename)
    plt.close()

# Tạo thư mục lưu ảnh (nếu chạy local)
!mkdir -p frames

# Di chuyển đến thư mục frames
os.chdir("frames")

# Khởi tạo robot và mô phỏng
robot = DifferentialRobot(start_pos=(1.0, 1.0), width=0.1)
dt = 0.05
total_time = 5  # giây
frames = int(total_time / dt)

for i in range(frames):
    # Điều chỉnh vận tốc tay trái và phải theo thời gian
    robot.vl += 0.0002  # tùy chỉnh logic để tạo chuyển động
    robot.vr += 0.0001
    robot.update(dt)
    draw_frame(robot, i)

# Quay lại thư mục gốc
os.chdir("..")


In [3]:
# Dùng OpenCV để ghép ảnh thành video
img_array = []
for i in range(frames):
    filename = f"frames/frame_{i:04d}.png"
    img = cv2.imread(filename)
    height, width, _ = img.shape
    img_array.append(img)

out = cv2.VideoWriter('robot_simulation.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      10, (width, height))

for img in img_array:
    out.write(img)
out.release()


In [4]:
from IPython.display import Video

Video("robot_simulation.mp4", embed=True)
